In [12]:
import pandas as pd
import tkinter as tk
import math
from tkinter import ttk
from tkinter import messagebox
from prettytable import PrettyTable  # Import PrettyTable
import os

# Define your CSV file paths here (use raw strings or double backslashes)
swl_csv = 'excavator_swl.csv'  # Ensure this file exists
bucket_csv = 'bucket_data.csv'  # Make sure this file exists
bhc_bucket_csv = 'bhc_bucket_data.csv'  # Make sure this file exists
dump_truck_csv = 'dump_trucks.csv'  # Path to dump truck CSV

# Step 1: Load Datasets
def load_bucket_data(bucket_csv):
    return pd.read_csv(bucket_csv)

def load_bhc_bucket_data(bhc_bucket_csv):
    return pd.read_csv(bhc_bucket_csv)

def load_dump_truck_data(dump_truck_csv):
    return pd.read_csv(dump_truck_csv)
    
# Loading the dump truck data
dump_truck_data = load_dump_truck_data(dump_truck_csv)

def load_excavator_swl_data(swl_csv):
    swl_data = pd.read_csv(swl_csv)
    swl_data['boom_length'] = pd.to_numeric(swl_data['boom_length'], errors='coerce')
    swl_data['arm_length'] = pd.to_numeric(swl_data['arm_length'], errors='coerce')
    swl_data['CWT'] = pd.to_numeric(swl_data['CWT'], errors='coerce')
    swl_data['shoe_width'] = pd.to_numeric(swl_data['shoe_width'], errors='coerce')
    swl_data['reach'] = pd.to_numeric(swl_data['reach'], errors='coerce')
    return swl_data

# Function to check if two floating-point numbers are "close enough"
def is_close(a, b, tolerance=0.01):
    return abs(a - b) <= tolerance

# Step 2: Dropdown User Inputs (using tkinter)
def get_user_inputs_gui(swl_data):
    # Create the main window
    root = tk.Tk()
    root.title("Excavator and Dump Truck Selection")

    # Define variables to hold user input
    make_var = tk.StringVar()
    model_var = tk.StringVar()
    boom_length_var = tk.DoubleVar()
    arm_length_var = tk.DoubleVar()
    cwt_var = tk.IntVar()
    shoe_width_var = tk.IntVar()
    reach_var = tk.DoubleVar()
    material_density_var = tk.DoubleVar()
    quick_hitch_weight_var = tk.DoubleVar()
    current_bucket_size_var = tk.DoubleVar()
    current_bucket_weight_var = tk.DoubleVar()
    machine_swings_per_minute_var = tk.DoubleVar()
    select_bhc_var = tk.BooleanVar()  # Checkbox variable for BHC buckets


    # Create frames for better layout
    frame1 = ttk.Frame(root, padding="10")
    frame1.grid(row=0, column=0)
    frame2 = ttk.Frame(root, padding="10")
    frame2.grid(row=1, column=0)
    
    # Create frame for dump truck inputs
    frame3 = ttk.Frame(root, padding="10")
    frame3.grid(row=2, column=0)

    # Dropdowns for Excavator Make and Model
    ttk.Label(frame1, text="Select Excavator Make:").grid(column=0, row=0)
    make_dropdown = ttk.Combobox(frame1, textvariable=make_var, values=swl_data['make'].unique().tolist())
    make_dropdown.grid(column=1, row=0)

    ttk.Label(frame1, text="Select Excavator Model:").grid(column=0, row=1)
    model_dropdown = ttk.Combobox(frame1, textvariable=model_var)
    model_dropdown.grid(column=1, row=1)

    # Dump Truck Dropdowns
    ttk.Label(frame3, text="Select Dump Truck Brand:").grid(column=0, row=0)
    truck_brand_var = tk.StringVar()
    truck_type_var = tk.StringVar()
    truck_model_var = tk.StringVar()
    truck_payload_var = tk.DoubleVar()

    truck_brand_dropdown = ttk.Combobox(frame3, textvariable=truck_brand_var, values=dump_truck_data['brand'].unique().tolist())
    truck_brand_dropdown.grid(column=1, row=0)

    ttk.Label(frame3, text="Select Dump Truck Type:").grid(column=0, row=1)
    truck_type_dropdown = ttk.Combobox(frame3, textvariable=truck_type_var)
    truck_type_dropdown.grid(column=1, row=1)

    ttk.Label(frame3, text="Select Dump Truck Model:").grid(column=0, row=2)
    truck_model_dropdown = ttk.Combobox(frame3, textvariable=truck_model_var)
    truck_model_dropdown.grid(column=1, row=2)

    ttk.Label(frame3, text="Select Dump Truck Payload (tons):").grid(column=0, row=3)
    truck_payload_dropdown = ttk.Combobox(frame3, textvariable=truck_payload_var)
    truck_payload_dropdown.grid(column=1, row=3)

    # Function to update excavator model dropdown based on selected make
    def update_model_dropdown(*args):
        selected_make = make_var.get()
        filtered_models = swl_data[swl_data['make'] == selected_make]['model'].unique().tolist()
        model_dropdown['values'] = filtered_models
        model_dropdown.set('')  # Reset model selection

    # Function to update excavator parameters based on selected model
    def update_parameters_dropdown(*args):
        selected_model = model_var.get()
        if selected_model:  # Ensure a model is selected
            filtered_data = swl_data[swl_data['model'] == selected_model]
            boom_length_dropdown['values'] = filtered_data['boom_length'].unique().tolist()
            arm_length_dropdown['values'] = filtered_data['arm_length'].unique().tolist()
            cwt_dropdown['values'] = filtered_data['CWT'].unique().tolist()
            shoe_width_dropdown['values'] = filtered_data['shoe_width'].unique().tolist()
            reach_dropdown['values'] = filtered_data['reach'].unique().tolist()
            # Reset selections
            boom_length_var.set(0)
            arm_length_var.set(0)
            cwt_var.set(0)
            shoe_width_var.set(0)
            reach_var.set(0)

    # Function to update dump truck types based on selected brand
    def update_truck_type_dropdown(*args):
        selected_brand = truck_brand_var.get()
        filtered_types = dump_truck_data[dump_truck_data['brand'] == selected_brand]['type'].unique().tolist()
        truck_type_dropdown['values'] = filtered_types
        truck_type_dropdown.set('')  # Reset type selection

    # Function to update dump truck models based on selected type
    def update_truck_model_dropdown(*args):
        selected_brand = truck_brand_var.get()
        selected_type = truck_type_var.get()
        filtered_models = dump_truck_data[(dump_truck_data['brand'] == selected_brand) & 
                                          (dump_truck_data['type'] == selected_type)]['model'].unique().tolist()
        truck_model_dropdown['values'] = filtered_models
        truck_model_dropdown.set('')  # Reset model selection

    # Function to update dump truck payload based on selected model
    def update_truck_payload_dropdown(*args):
        selected_model = truck_model_var.get()
        filtered_payloads = dump_truck_data[dump_truck_data['model'] == selected_model]['payload'].unique().tolist()
        truck_payload_dropdown['values'] = filtered_payloads
        truck_payload_dropdown.set('')  # Reset payload selection

    # Bind the dropdowns to update their values
    make_dropdown.bind('<<ComboboxSelected>>', update_model_dropdown)
    model_dropdown.bind('<<ComboboxSelected>>', update_parameters_dropdown)
    truck_brand_dropdown.bind('<<ComboboxSelected>>', update_truck_type_dropdown)
    truck_type_dropdown.bind('<<ComboboxSelected>>', update_truck_model_dropdown)
    truck_model_dropdown.bind('<<ComboboxSelected>>', update_truck_payload_dropdown)

    # Boom Length, Arm Length, CWT, Shoe Width, Reach Dropdowns
    ttk.Label(frame2, text="Select Boom Length (m):").grid(column=0, row=2)
    boom_length_dropdown = ttk.Combobox(frame2, textvariable=boom_length_var)
    boom_length_dropdown.grid(column=1, row=2)

    ttk.Label(frame2, text="Select Arm Length (m):").grid(column=0, row=3)
    arm_length_dropdown = ttk.Combobox(frame2, textvariable=arm_length_var)
    arm_length_dropdown.grid(column=1, row=3)

    ttk.Label(frame2, text="Select Counterweight (CWT in kg):").grid(column=0, row=4)
    cwt_dropdown = ttk.Combobox(frame2, textvariable=cwt_var)
    cwt_dropdown.grid(column=1, row=4)

    ttk.Label(frame2, text="Select Shoe Width (mm):").grid(column=0, row=5)
    shoe_width_dropdown = ttk.Combobox(frame2, textvariable=shoe_width_var)
    shoe_width_dropdown.grid(column=1, row=5)

    ttk.Label(frame2, text="Select Reach (m):").grid(column=0, row=6)
    reach_dropdown = ttk.Combobox(frame2, textvariable=reach_var)
    reach_dropdown.grid(column=1, row=6)

    # Additional Inputs
    ttk.Label(frame2, text="Material Density (kg/m³):").grid(column=0, row=7)
    ttk.Entry(frame2, textvariable=material_density_var).grid(column=1, row=7)

    # Additional Inputs
    ttk.Label(frame2, text="Quick Hitch Weight (kg):").grid(column=0, row=8)
    ttk.Entry(frame2, textvariable=quick_hitch_weight_var).grid(column=1, row=8)

    ttk.Label(frame2, text="Current Bucket Size (m³):").grid(column=0, row=9)
    ttk.Entry(frame2, textvariable=current_bucket_size_var).grid(column=1, row=9)

    ttk.Label(frame2, text="Current Bucket Weight (kg):").grid(column=0, row=10)
    ttk.Entry(frame2, textvariable=current_bucket_weight_var).grid(column=1, row=10)

    ttk.Label(frame2, text="Machine Swings per Minute:").grid(column=0, row=11)
    ttk.Entry(frame2, textvariable=machine_swings_per_minute_var).grid(column=1, row=11)

    bhc_checkbox = ttk.Checkbutton(frame1, text="Select from BHC buckets only", variable=select_bhc_var)
    bhc_checkbox.grid(column=0, row=12, sticky="w")  # Adjust row as needed

    # Submit function
    def submit():
        user_data = {
            'make': make_var.get(),
            'model': model_var.get(),
            'boom_length': boom_length_var.get(),
            'arm_length': arm_length_var.get(),
            'cwt': cwt_var.get(),
            'shoe_width': shoe_width_var.get(),
            'reach': reach_var.get(),
            'material_density': (material_density_var.get()),
            'quick_hitch_weight': (quick_hitch_weight_var.get()),
            'current_bucket_size': (current_bucket_size_var.get()), 
            'current_bucket_weight': (current_bucket_weight_var.get()), 
            'dump_truck_payload': float(truck_payload_var.get()),  # Ensure this variable is set
            'machine_swings_per_minute': (machine_swings_per_minute_var.get()) 
        }
        
        # Determine which bucket CSV to use based on checkbox state
        selected_bucket_csv = bhc_bucket_csv if select_bhc_var.get() else bucket_csv
        user_data['bucket_data'] = load_bucket_data(selected_bucket_csv)  # Load selected bucket data
        
        # Close the GUI
        root.destroy()
        
        # Process the collected user data
        process_user_data(user_data)

    # Submit button
    ttk.Button(root, text="Submit", command=submit).grid(column=0, row=10)

    root.mainloop()  # Start the GUI event loop

# Step 3: Find Matching Excavator SWL based on User Inputs
def find_matching_swl(user_data, swl_data):
    matching_excavator = swl_data[
        (swl_data['make'] == user_data['make']) &
        (swl_data['model'] == user_data['model']) &
        (swl_data['CWT'] == user_data['cwt']) &
        (swl_data['shoe_width'] == user_data['shoe_width']) &
        (swl_data['reach'].apply(lambda x: is_close(x, user_data['reach']))) &
        (swl_data['boom_length'].apply(lambda x: is_close(x, user_data['boom_length']))) &
        (swl_data['arm_length'].apply(lambda x: is_close(x, user_data['arm_length'])))
    ]

    if matching_excavator.empty:
        print("No matching excavator configuration found!")
        return None

    swl = matching_excavator.iloc[0]['swl']
    return swl

# Step 4: Calculate Load for a Given Bucket
def calculate_bucket_load(bucket_size, material_density):
    return bucket_size * material_density

def select_optimal_bucket(user_data, bucket_data, swl):
    current_bucket_size = user_data['current_bucket_size']
    optimal_bucket = None
    highest_bucket_size = 0

    for index, bucket in bucket_data.iterrows():
        # Calculate the load based on the bucket size and material density
        bucket_load = calculate_bucket_load(bucket['bucket_size'], user_data['material_density'])
        # Calculate the total bucket weight
        total_bucket_weight = user_data['quick_hitch_weight'] + bucket_load + bucket['bucket_weight']

        # Check if this bucket is optimal and within SWL
        if total_bucket_weight <= swl and bucket['bucket_size'] > highest_bucket_size:
            highest_bucket_size = bucket['bucket_size']
            percentage_increase = ((highest_bucket_size - current_bucket_size) / current_bucket_size) * 100
            optimal_bucket = {
                'bucket_name': bucket['bucket_name'],
                'bucket_size': highest_bucket_size,
                'percentage_increase': percentage_increase,
                'bucket_weight': bucket['bucket_weight'],
                'total_bucket_weight': total_bucket_weight
            }

    if optimal_bucket:
        # Display the bucket selection info
        messagebox.showinfo("Optimal Bucket", 
                            f"Good news! Ontrac could increase your bucket capacity by {optimal_bucket['percentage_increase']:.2f}% with an XMOR-{optimal_bucket['bucket_name']}.\n\n"
                            f"Calculations based on a: {user_data['make']} {user_data['model']}, with a {user_data['boom_length']}m boom and {user_data['arm_length']}m arm, "
                            f"{user_data['cwt']}kg CWT, and {user_data['shoe_width']}mm shoe width, with a SWL of {swl} kg at {user_data['reach']}m."
                            f"\n\nBucket Weight (from CSV): {optimal_bucket['bucket_weight']}kg"
                            f"\n\nTotal Suspended Load Calculation: \n(bucket + load + quick hitch if applicable) = {optimal_bucket['total_bucket_weight']}kg")

        # Return both weights for confirmation
        return optimal_bucket['total_bucket_weight'], optimal_bucket['bucket_name'], optimal_bucket['bucket_size'], optimal_bucket['percentage_increase']

    print("No suitable bucket found within SWL limits.")
    return None, None, None


# Step 6: Process User Data and Display Table
def process_user_data(user_data):
    bucket_data = user_data['bucket_data']
    swl_data = load_excavator_swl_data(swl_csv)

    swl = find_matching_swl(user_data, swl_data)
    if swl is None:
        return

    optimal_bucket_weight, optimal_bucket_name, optimal_bucket_size, percentage_increase = select_optimal_bucket(user_data, bucket_data, swl)
    
    if optimal_bucket_name:
        old_capacity = user_data['current_bucket_size']
        new_capacity = optimal_bucket_size
        old_payload = calculate_bucket_load(old_capacity, user_data['material_density'])
        new_payload = calculate_bucket_load(new_capacity, user_data['material_density'])

        # Initial dump truck payload
        dump_truck_payload = user_data['dump_truck_payload'] * 1000
        machine_swings_per_minute = user_data['machine_swings_per_minute']

        # Total suspended load
        old_total_load = old_payload + user_data['current_bucket_weight'] + user_data['quick_hitch_weight']
        new_total_load = optimal_bucket_weight

        # Adjust payload to achieve whole or near-whole swings for the new payload
        def adjust_payload_for_new_bucket(dump_truck_payload, new_payload):
            max_payload = dump_truck_payload * 1.10
            increment = dump_truck_payload * 0.001  # Fine adjustment increments

            # Try to achieve swing values within ±0.1 first
            while dump_truck_payload <= max_payload:
                swings_to_fill_truck_new = dump_truck_payload / new_payload
                if abs(swings_to_fill_truck_new - round(swings_to_fill_truck_new)) <= 0.1:
                    return dump_truck_payload, swings_to_fill_truck_new

                dump_truck_payload += increment

            # If ±0.1 is not possible, allow ±0.2 tolerance
            dump_truck_payload = user_data['dump_truck_payload'] * 1000  # Reset payload for ±0.2 tolerance
            while dump_truck_payload <= max_payload:
                swings_to_fill_truck_new = dump_truck_payload / new_payload
                if abs(swings_to_fill_truck_new - round(swings_to_fill_truck_new)) <= 0.2:
                    return dump_truck_payload, swings_to_fill_truck_new

                dump_truck_payload += increment

            # Return the closest achievable value
            return dump_truck_payload, swings_to_fill_truck_new

        # Adjust the dump truck payload based on the new bucket payload only
        dump_truck_payload, swings_to_fill_truck_new = adjust_payload_for_new_bucket(dump_truck_payload, new_payload)
        swings_to_fill_truck_old = dump_truck_payload / old_payload

        # Time to fill truck in minutes
        time_to_fill_truck_old = math.ceil(swings_to_fill_truck_old) / machine_swings_per_minute
        time_to_fill_truck_new = swings_to_fill_truck_new / machine_swings_per_minute

        # Average number of trucks per hour at 75% efficiency
        avg_trucks_per_hour_old = (60 / time_to_fill_truck_old) * 0.75 if time_to_fill_truck_old > 0 else 0
        avg_trucks_per_hour_new = (60 / time_to_fill_truck_new) * 0.75 if time_to_fill_truck_new > 0 else 0

        # Swings per hour
        swings_per_hour_old = math.ceil(swings_to_fill_truck_old) * avg_trucks_per_hour_old
        swings_per_hour_new = swings_to_fill_truck_new * avg_trucks_per_hour_new

        # Total swings per hour
        total_swings_per_hour = 60 * machine_swings_per_minute

        # Production (t/hr)
        total_tonnage_per_hour_old = total_swings_per_hour * old_capacity * user_data['material_density'] / 1000
        total_tonnage_per_hour_new = total_swings_per_hour * new_capacity * user_data['material_density'] / 1000

        # Production (t/hr)
        tonnage_per_hour_old = avg_trucks_per_hour_old * dump_truck_payload /1000
        tonnage_per_hour_new = avg_trucks_per_hour_new * dump_truck_payload /1000

        # Assuming 1800 swings in a day
        total_m3_per_day_old = 1000 * old_capacity
        total_m3_per_day_new = 1000 * new_capacity

        # Total tonnage per day
        total_tonnage_per_day_old = total_m3_per_day_old * user_data['material_density'] / 1000
        total_tonnage_per_day_new = total_m3_per_day_new * user_data['material_density'] / 1000

        # Total number of trucks per day
        total_trucks_per_day_old = total_tonnage_per_day_old / dump_truck_payload * 1000
        total_trucks_per_day_new = total_tonnage_per_day_new / dump_truck_payload * 1000

        # Now continue with the rest of the code as needed
        total_trucks_per_day_new = total_tonnage_per_day_new / dump_truck_payload * 1000

# Data for comparison table
        data = {
        'Description': [
            'Side-By-Side Bucket Comparison', 'Capacity (m³)', 'Material Density (kg/m³)', 'Bucket Payload (kg)', 
            'Total Suspended Load (kg)', '', 
            'Loadout Productivity & Truck Pass Simulation', 'Dump Truck Payload (kg)', 'Avg No. Swings to Fill Truck', 
            'Time to Fill Truck (min)', 'Avg Trucks/Hour @ 75% eff', 'Total Swings/Hour', '', 
            'Swings Per Day Side-By-Side Simulation', 'Total Tonnage/hr', 'Total m³/Day', 
            'Total Tonnage/Day', 'Total Trucks/Day', '', 
            'Improved Cycle Time Simulation', 'Total Tonnage/hr', 'Total m³/Day', 
            'Total Tonnage/Day', 'Total Trucks/Day'
        ],
        'OLD Bucket': [
            '', f"{old_capacity:.1f}", f"{user_data['material_density']:.1f}", f"{old_payload:.1f}", 
            f"{old_total_load:.1f}", '', 
            '', f"{dump_truck_payload:.1f}", f"{swings_to_fill_truck_old:.1f}", 
            f"{time_to_fill_truck_old:.1f}", f"{avg_trucks_per_hour_old:.1f}", f"{swings_per_hour_old:.1f}", '', 
            '', f"{total_tonnage_per_hour_old:.1f}", f"{total_m3_per_day_old:.1f}", 
            f"{total_tonnage_per_day_old:.1f}", f"{total_trucks_per_day_old:.1f}", '', 
            '', f"{total_tonnage_per_hour_old:.1f}", f"{total_m3_per_day_old:.1f}", 
            f"{total_tonnage_per_day_old:.1f}", f"{total_trucks_per_day_old:.1f}"
        ],
        'New Bucket': [
            '', f"{new_capacity:.1f}", f"{user_data['material_density']:.1f}", f"{new_payload:.1f}", 
            f"{new_total_load:.1f}", '', 
            '', f"{dump_truck_payload:.1f}", f"{swings_to_fill_truck_new:.1f}", 
            f"{time_to_fill_truck_new:.1f}", f"{avg_trucks_per_hour_new:.1f}", f"{swings_per_hour_new:.1f}", '', 
            '', f"{total_tonnage_per_hour_new:.1f}", f"{total_m3_per_day_new:.1f}", 
            f"{total_tonnage_per_day_new:.1f}", f"{total_trucks_per_day_new:.1f}", '', 
            '', f"{1.1 * total_tonnage_per_hour_new:.1f}", f"{1.1 * total_m3_per_day_new:.1f}", 
            f"{1.1 * total_tonnage_per_day_new:.1f}", f"{1.1 * total_trucks_per_day_new:.1f}"
        ],
        'Difference': [
            '', f"{new_capacity - old_capacity:.1f}", '-', f"{new_payload - old_payload:.1f}", 
            f"{new_total_load - old_total_load:.1f}", '', 
            '', '-', f"{swings_to_fill_truck_new - swings_to_fill_truck_old:.1f}", 
            f"{time_to_fill_truck_new - time_to_fill_truck_old:.1f}", 
            f"{avg_trucks_per_hour_new - avg_trucks_per_hour_old:.1f}", '-', '', 
            '', f"{total_tonnage_per_hour_new - total_tonnage_per_hour_old:.1f}", 
            f"{total_m3_per_day_new - total_m3_per_day_old:.1f}", 
            f"{total_tonnage_per_day_new - total_tonnage_per_day_old:.1f}", 
            f"{total_trucks_per_day_new - total_trucks_per_day_old:.1f}", '', 
            '', f"{(1.1 * total_tonnage_per_hour_new - total_tonnage_per_hour_old):.1f}", 
            f"{(1.1 * total_m3_per_day_new - total_m3_per_day_old):.1f}", 
            f"{(1.1 * total_tonnage_per_day_new - total_tonnage_per_day_old):.1f}", 
            f"{(1.1 * total_trucks_per_day_new - total_trucks_per_day_old):.1f}"
        ],
        '% Difference': [
            '', f"{percentage_increase:.1f}%", '-', f"{(new_payload - old_payload) / old_payload * 100:.1f}%", 
            f"{(new_total_load - old_total_load) / old_total_load * 100:.1f}%", '', 
            '', '-', f"{(swings_to_fill_truck_new - swings_to_fill_truck_old) / swings_to_fill_truck_old * 100:.1f}%", 
            f"{(time_to_fill_truck_new - time_to_fill_truck_old) / time_to_fill_truck_old * 100:.1f}%", 
            f"{(avg_trucks_per_hour_new - avg_trucks_per_hour_old) / avg_trucks_per_hour_old * 100:.1f}%", '-', '', 
            '', f"{(total_tonnage_per_hour_new - total_tonnage_per_hour_old) / total_tonnage_per_hour_old * 100:.1f}%", 
            f"{(total_m3_per_day_new - total_m3_per_day_old) / total_m3_per_day_old * 100:.1f}%", 
            f"{(total_tonnage_per_day_new - total_tonnage_per_day_old) / total_tonnage_per_day_old * 100:.1f}%", 
            f"{(total_trucks_per_day_new - total_trucks_per_day_old) / total_trucks_per_day_old * 100:.1f}%", '', 
            '', f"{(1.1 * total_tonnage_per_hour_new - total_tonnage_per_hour_old) / total_tonnage_per_hour_old * 100:.1f}%", 
            f"{(1.1 * total_m3_per_day_new - total_m3_per_day_old) / total_m3_per_day_old * 100:.1f}%", 
            f"{(1.1 * total_tonnage_per_day_new - total_tonnage_per_day_old) / total_tonnage_per_day_old * 100:.1f}%", 
            f"{(1.1 * total_trucks_per_day_new - total_trucks_per_day_old) / total_trucks_per_day_old * 100:.1f}%"
        ]
    }

        # Create DataFrame and save to Excel
        df = pd.DataFrame(data)
        excel_filename = "Productivity_Study.xlsx"
        df.to_excel(excel_filename, index=False)

# Main execution
if __name__ == "__main__":
    swl_data = load_excavator_swl_data(swl_csv)  # Load your SWL data here
    get_user_inputs_gui(swl_data)


SyntaxError: unterminated string literal (detected at line 14) (973300177.py, line 14)